# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_csv = "../output_data/cities.csv"
clean_city_data_df = pd.read_csv(cities_csv)
clean_city_data_df = clean_city_data_df.set_index("City_ID")
clean_city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Tasiilaq,65.6145,-37.6368,26.91,67,0,8.01,GL,1604350800
1,Am Timan,11.0297,20.2827,81.81,34,29,2.55,TD,1604350800
2,Evensk,61.9500,159.2333,25.47,97,99,5.46,RU,1604350800
3,Wollongong,-34.4333,150.8833,61.52,80,24,1.79,AU,1604350800
4,Punta Arenas,-53.1500,-70.9167,56.21,56,87,2.42,CL,1604350800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
latlng_df = clean_city_data_df[["Lat", "Lng"]].astype(float)
latlng_df.head()
humidity_df = clean_city_data_df["Humidity"].astype(float)
humidity_df.head()

City_ID
0    67.0
1    34.0
2    97.0
3    80.0
4    56.0
Name: Humidity, dtype: float64

In [11]:
fig = gmaps.figure(center=(50.0, 0.0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(latlng_df, weights=humidity_df, max_intensity=100, 
                                    point_radius=3.0, dissipating=False, opacity=.5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * A max temperature lower than 80 degrees but higher than 70
    * Wind speed less than 10 mph.
    * Zero cloudiness.
* Drop any rows will null values.

In [14]:
hotel_df = clean_city_data_df.loc[((clean_city_data_df["Max Temp"] < 80) & 
                                   (clean_city_data_df["Max Temp"] > 70) &
                                   (clean_city_data_df["Wind Speed"] < 10) &
                                   (clean_city_data_df["Cloudiness"] == 0)), :]
hotel_df = hotel_df.dropna(how="any")
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
229,Alice Springs,-23.7000,133.8833,74.62,20,0,7.20,AU,1604350800
273,Cruz Alta,-28.6386,-53.6064,73.40,44,0,8.30,BR,1604350800
310,Salalah,17.0151,54.0924,75.40,55,0,2.62,OM,1604350800
410,Misratah,32.3754,15.0925,70.41,60,0,4.43,LY,1604350800
447,Rodolfo Sánchez Taboada,31.7167,-116.5667,77.54,66,0,7.58,MX,1604350800
452,Reconquista,-29.1500,-59.6500,78.67,38,0,8.23,AR,1604350800
464,Tiznit Province,29.5833,-9.5000,70.32,36,0,2.13,MA,1604350800
471,Agadez,19.7500,10.2500,71.01,24,0,7.20,NE,1604350800
534,Dholka,22.7167,72.4667,74.14,49,0,6.51,IN,1604350800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df["Hotel Name"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [45]:
# API Testing code
# test_url = f"{base_url}?location=-23.7000,133.8833&radius=5000&types=hotel&key={g_key}"
# test_response = requests.get(test_url)
# test_data = test_response.json()
# pprint(test_data["results"])
# print("--->" + test_data["results"][0]['name'])

# Iterate over the rows in the dataframe
for index, row in hotel_df.iterrows():

    # Grab the city information from the DataFrame Row
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    # print(f"City={city} lat={lat} lng={lng}")
    
    # Set up the parameters
    params = {
              "location": f"{lat},{lng}",
              "types": "hotel",
              "radius": 5000,
              "key": g_key,
             }
    
    # Make API request
    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Get the results
    results = data["results"]
    
    try:
        name = results[0]['name']
        print(f"Nearest hotel to {city} is {name}.")        
        hotel_df.loc[index, "Hotel Name"] = name
        
    except (KeyError, IndexError):
        print(f"Error with city={city}, skipping...")
        hotel_df.loc[index, "Hotel Name"] = "NaN"
    
    hotel_df.head()

Nearest hotel to Alice Springs is Alice Springs.
Nearest hotel to Cruz Alta is Cruz Alta.
Nearest hotel to Salalah is Salalah.
Nearest hotel to Misratah is Misrata.
Nearest hotel to Rodolfo Sánchez Taboada is El Maneadero.
Nearest hotel to Reconquista is Reconquista.
Nearest hotel to Tiznit Province is Ouijjane.
Error with city=Agadez, skipping...
Nearest hotel to Dholka is Dholka.
Nearest hotel to Ţahţā is Tahta.
Nearest hotel to Olavarría is Olavarría.
Nearest hotel to Pasni is Pasni.


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))